In [2]:
!which python

/Users/pushkalm11/miniconda3/envs/unrolling/bin/python


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

In [4]:
import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

In [5]:
import networkx as nx
import sys

In [6]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Flatten, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping

In [7]:
print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [7]:
X_train = loadmat('./Dataset/X_train.mat')['X']
loadmat('./Dataset/Global.mat')['A']

N, T = X_train.shape

In [8]:
dt = np.zeros([T - 1, T])

for i in range(T - 1):
    dt[i, i] = -1
    dt[i, i - 1] = 1

d = dt.T
d = np.concatenate((d, np.zeros([T, 1])), axis = 1) 
d[T - 1, T - 1] = -1
d = d.T
d[T - 1, T - 1] = 1 
d[-1, -2] = 1
d[0, -1] = 0
d = tf.convert_to_tensor(d, dtype = "float32")
print(d)

tf.Tensor(
[[-1.  0.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0.  1.  1.]], shape=(500, 500), dtype=float32)


2023-10-07 00:56:40.256698: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-07 00:56:40.258348: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
def adj_to_laplacian(AdjL):
    tmp = np.array(AdjL, dtype = "float32")
    return np.diag(np.sum(tmp, axis = 0)) - tmp

In [10]:
psi_cost = 0
test_list = []

In [11]:
# A - Predicted, B - Target matrix
def costfunc(A, B):
    # (y_true, y_pred)
    # print(A.shape)
    # print(B.shape)
    return (tf.norm(tf.multiply(psi_cost, (A - B)))) ** 2

# def costfunc(A, B):
#     return (tf.norm(tf.multiply(psi_cost, (A - B))) ** 2) + (reg * tf.norm(HLG @ B @ GLT) ** 2 / 2)

def mse_unknown(A, B):

    global psi_cost
    psi = psi_cost

    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B)))) ** 2 / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):

    global psi_cost 
    psi = psi_cost

    mse_kn = (LA.norm(np.multiply(psi, (X_train - B)))) ** 2 / np.sum(psi.numpy().flatten())
    
    global mse_known_list
    mse_known_list.append(mse_kn)

    return mse_kn

In [12]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.00001, verbose = 1, patience = 10)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [13]:
class EldarModel(tf.keras.Model):
    
    def __init__(self, Laplacian, reg_init, loops_init, d_init, **kwargs):
        super(EldarModel, self).__init__(**kwargs)
        p_init = 20.0
        self.p1_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        self.p2_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        self.p3_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p3 = tf.Variable(initial_value = self.p3_init, trainable = True, name = "p3")
        self.p4_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p4 = tf.Variable(initial_value = self.p4_init, trainable = True, name = "p4")
        
        self.lap = Laplacian
        self.reg = reg_init
        self.loops = loops_init
        self.d_matrix = d_init
        
    def call(self, y):
        
        psi = y[0, :, T:]
        global psi_cost
        psi_cost = psi
        
        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0, :, : T]
        X_in = tf.convert_to_tensor(X_in, dtype = "float32")
        
        Y = X_in
        
        LG, reg, loop, d = self.lap, self.reg, self.loops, self.d_matrix
        
        LT = tf.matmul(tf.transpose(d), d)
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        HLG = tf.eye(tf.shape(LG)[0], dtype = "float32") 
        GLT = LT + (self.p1 * MP(LT, 2)) + (self.p2 * MP(LT, 3)) + (self.p3 * MP(LT, 4)) + (self.p4 * MP(LT, 5))
        
        Xk = tf.zeros_like(X_in)
        Zk = tf.zeros_like(X_in)
        Zk = -(tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT)))
        
        for i in range(loop):
            fdx_xk = tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT))
            fdx_zk = tf.multiply(psi, Zk) - Y + (reg * (HLG @ Zk @ GLT))
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
            
            Xk_1 = Xk - (tau * Zk)
            
            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (reg * (HLG @ Xk_1 @ GLT))
            
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)
            
            Zk_1 = (gamma * Zk) - fdx_xk_1
            
            Xk = Xk_1
            Zk = Zk_1
        
        global test_list
        test_list.append(Xk_1.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [14]:
no_of_psi = 10
Laplacian = adj_to_laplacian(Adj_matrix)

sensing_ratio = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5]

# reg_list = np.load("./Outputs/eldar_GT_reg.npy")
# loops_list = np.load("./Outputs/eldar_GT_loops.npy")

reg_list = [1.0] * len(sensing_ratio)
loops_list = [50] * len(sensing_ratio)

all_psi = []

mse_known_list = []
mse_unknown_list = []

learned_variables = []

lr_decay_factor = 1.04

In [15]:
for i_sen, rem in enumerate(sensing_ratio):
    
    print(f"\n\n###########################################################################################")
    print(f"################################### Sensing Ratio: {rem * 100}% ###################################")
    print(f"###########################################################################################\n\n")
    
    if i_sen == 0:
        M = int(rem * T)
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2*T])
    
        for i in range(no_of_psi):
            psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
            np.random.shuffle(psi_k)
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
    else:
        previous_psi = all_psi[-no_of_psi : ]
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])
        
        rem_diff = int(N * (rem - sensing_ratio[i_sen - 1]) * T)
        
        for i in range(no_of_psi):
            psi_k_tmp = np.array(previous_psi[i]).flatten()
            ones_idx = np.where(psi_k_tmp == 1)
            
            ones_to_zero_idx = np.random.choice(ones_idx[0], size = rem_diff)
            psi_k_tmp[ones_to_zero_idx] = 0
            psi_k = psi_k_tmp
            
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
        
    reg_init = reg_list[i_sen]
    loops_init = loops_list[i_sen]
    
    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 1.0e-3, momentum = 0.05)
    
    eldar_model = EldarModel(Laplacian, reg_init, loops_init, d, name = 'EM')
    eldar_model.compile(optimizer = sgd_optimizer, loss = costfunc, 
                        metrics = [mse_known, mse_unknown], run_eagerly = True)
    eldar_model.fit(X_train_concatenated, X_train_missing, epochs = 100, 
                    callbacks = [lr_scheduler], batch_size = 1)
    eldar_model.summary()
    
    all_variables = eldar_model.variables
    
    graph_vars = [i.numpy()[0, 0] for i in all_variables]
    learned_variables.append(graph_vars)
    
    print(f"Learned Variables: {graph_vars}")



###########################################################################################
################################### Sensing Ratio: 1.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 7s 713ms/step - loss: 1010.3861 - mse_known: 0.1021 - mse_unknown: 0.1339 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 7s 712ms/step - loss: 1007.9266 - mse_known: 0.1018 - mse_unknown: 0.1337 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 7s 675ms/step - loss: 1005.3444 - mse_known: 0.1015 - mse_unknown: 0.1335 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 7s 714ms/step - loss: 1002.8798 - mse_known: 0.1013 - mse_unknown: 0.1333 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 7s 657ms/step - loss: 1000.3240 - mse_known: 0.1010 - mse_unknown: 0.1331 - lr: 0.0010
Epoch 6/100
10/10 [=

10/10 [==============================] - 8s 839ms/step - loss: 929.5828 - mse_known: 0.0939 - mse_unknown: 0.1274 - lr: 1.6461e-04
Epoch 57/100
10/10 [==============================] - 8s 823ms/step - loss: 929.0563 - mse_known: 0.0938 - mse_unknown: 0.1274 - lr: 1.5828e-04
Epoch 58/100
10/10 [==============================] - 8s 828ms/step - loss: 928.6675 - mse_known: 0.0938 - mse_unknown: 0.1273 - lr: 1.5219e-04
Epoch 59/100
10/10 [==============================] - 8s 845ms/step - loss: 928.2230 - mse_known: 0.0938 - mse_unknown: 0.1273 - lr: 1.4634e-04
Epoch 60/100
10/10 [==============================] - 8s 824ms/step - loss: 927.7903 - mse_known: 0.0937 - mse_unknown: 0.1272 - lr: 1.4071e-04
Epoch 61/100
10/10 [==============================] - 8s 762ms/step - loss: 927.4199 - mse_known: 0.0937 - mse_unknown: 0.1272 - lr: 1.3530e-04
Epoch 62/100
10/10 [==============================] - 8s 841ms/step - loss: 926.9619 - mse_known: 0.0936 - mse_unknown: 0.1272 - lr: 1.3010e-04
Epoch

10/10 [==============================] - 8s 783ms/step - loss: 975.2197 - mse_known: 0.1026 - mse_unknown: 0.1364 - lr: 0.0010
Epoch 9/100
10/10 [==============================] - 8s 823ms/step - loss: 972.8005 - mse_known: 0.1023 - mse_unknown: 0.1362 - lr: 0.0010
Epoch 10/100
10/10 [==============================] - 7s 738ms/step - loss: 970.1790 - mse_known: 0.1020 - mse_unknown: 0.1360 - lr: 0.0010
Epoch 11/100
10/10 [==============================] - 7s 754ms/step - loss: 967.7458 - mse_known: 0.1018 - mse_unknown: 0.1358 - lr: 9.6154e-04
Epoch 12/100
10/10 [==============================] - 7s 718ms/step - loss: 965.3431 - mse_known: 0.1015 - mse_unknown: 0.1356 - lr: 9.2456e-04
Epoch 13/100
10/10 [==============================] - 7s 745ms/step - loss: 963.0204 - mse_known: 0.1013 - mse_unknown: 0.1354 - lr: 8.8900e-04
Epoch 14/100
10/10 [==============================] - 7s 708ms/step - loss: 960.8301 - mse_known: 0.1011 - mse_unknown: 0.1352 - lr: 8.5480e-04
Epoch 15/100
10/10

10/10 [==============================] - 7s 706ms/step - loss: 910.0175 - mse_known: 0.0957 - mse_unknown: 0.1310 - lr: 1.1566e-04
Epoch 66/100
10/10 [==============================] - 7s 730ms/step - loss: 909.7334 - mse_known: 0.0957 - mse_unknown: 0.1310 - lr: 1.1121e-04
Epoch 67/100
10/10 [==============================] - 7s 730ms/step - loss: 909.4302 - mse_known: 0.0957 - mse_unknown: 0.1309 - lr: 1.0693e-04
Epoch 68/100
10/10 [==============================] - 7s 715ms/step - loss: 909.1555 - mse_known: 0.0956 - mse_unknown: 0.1309 - lr: 1.0282e-04
Epoch 69/100
10/10 [==============================] - 7s 737ms/step - loss: 908.9025 - mse_known: 0.0956 - mse_unknown: 0.1309 - lr: 9.8863e-05
Epoch 70/100
10/10 [==============================] - 7s 693ms/step - loss: 908.5384 - mse_known: 0.0956 - mse_unknown: 0.1309 - lr: 9.5060e-05
Epoch 71/100
10/10 [==============================] - 7s 710ms/step - loss: 908.3561 - mse_known: 0.0955 - mse_unknown: 0.1308 - lr: 9.1404e-05
Epoch

10/10 [==============================] - 8s 847ms/step - loss: 930.8868 - mse_known: 0.1032 - mse_unknown: 0.1418 - lr: 7.5992e-04
Epoch 18/100
10/10 [==============================] - 8s 827ms/step - loss: 929.0234 - mse_known: 0.1030 - mse_unknown: 0.1417 - lr: 7.3069e-04
Epoch 19/100
10/10 [==============================] - 8s 822ms/step - loss: 927.2977 - mse_known: 0.1028 - mse_unknown: 0.1415 - lr: 7.0259e-04
Epoch 20/100
10/10 [==============================] - 8s 837ms/step - loss: 925.5660 - mse_known: 0.1026 - mse_unknown: 0.1414 - lr: 6.7556e-04
Epoch 21/100
10/10 [==============================] - 8s 752ms/step - loss: 923.8353 - mse_known: 0.1024 - mse_unknown: 0.1412 - lr: 6.4958e-04
Epoch 22/100
10/10 [==============================] - 8s 834ms/step - loss: 922.2191 - mse_known: 0.1022 - mse_unknown: 0.1411 - lr: 6.2460e-04
Epoch 23/100
10/10 [==============================] - 8s 822ms/step - loss: 920.6544 - mse_known: 0.1021 - mse_unknown: 0.1409 - lr: 6.0057e-04
Epoch

...

Epoch 71/100
10/10 [==============================] - 8s 837ms/step - loss: 718.1785 - mse_known: 0.1174 - mse_unknown: 0.1697 - lr: 9.1404e-05
Epoch 72/100
10/10 [==============================] - 8s 827ms/step - loss: 718.0034 - mse_known: 0.1174 - mse_unknown: 0.1697 - lr: 8.7889e-05
Epoch 73/100
10/10 [==============================] - 8s 842ms/step - loss: 717.8549 - mse_known: 0.1174 - mse_unknown: 0.1696 - lr: 8.4508e-05
Epoch 74/100
10/10 [==============================] - 9s 838ms/step - loss: 717.6407 - mse_known: 0.1173 - mse_unknown: 0.1696 - lr: 8.1258e-05
Epoch 75/100
10/10 [==============================] - 9s 863ms/step - loss: 717.5051 - mse_known: 0.1173 - mse_unknown: 0.1696 - lr: 7.8133e-05
Epoch 76/100
10/10 [==============================] - 8s 845ms/step - loss: 717.3964 - mse_known: 0.1173 - mse_unknown: 0.1696 - lr: 7.5128e-05
Epoch 77/100
10/10 [==============================] - 8s 832ms/step - loss: 717.3125 - mse_known: 0.1173 - mse_unknown: 0.1696 - lr: 7.2

10/10 [==============================] - 8s 850ms/step - loss: 703.5638 - mse_known: 0.1248 - mse_unknown: 0.1783 - lr: 6.0057e-04
Epoch 24/100
10/10 [==============================] - 8s 833ms/step - loss: 702.6227 - mse_known: 0.1246 - mse_unknown: 0.1782 - lr: 5.7748e-04
Epoch 25/100
10/10 [==============================] - 9s 848ms/step - loss: 701.8136 - mse_known: 0.1245 - mse_unknown: 0.1781 - lr: 5.5526e-04
Epoch 26/100
10/10 [==============================] - 8s 839ms/step - loss: 701.0501 - mse_known: 0.1244 - mse_unknown: 0.1780 - lr: 5.3391e-04
Epoch 27/100
10/10 [==============================] - 8s 846ms/step - loss: 700.2059 - mse_known: 0.1242 - mse_unknown: 0.1779 - lr: 5.1337e-04
Epoch 28/100
10/10 [==============================] - 9s 870ms/step - loss: 699.4933 - mse_known: 0.1241 - mse_unknown: 0.1779 - lr: 4.9363e-04
Epoch 29/100
10/10 [==============================] - 9s 850ms/step - loss: 698.7310 - mse_known: 0.1240 - mse_unknown: 0.1778 - lr: 4.7464e-04
Epoch

Epoch 80/100
10/10 [==============================] - 8s 839ms/step - loss: 682.4371 - mse_known: 0.1211 - mse_unknown: 0.1759 - lr: 6.4219e-05
Epoch 81/100
10/10 [==============================] - 8s 833ms/step - loss: 682.3782 - mse_known: 0.1211 - mse_unknown: 0.1758 - lr: 6.1749e-05
Epoch 82/100
10/10 [==============================] - 8s 842ms/step - loss: 682.2139 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.9374e-05
Epoch 83/100
10/10 [==============================] - 8s 843ms/step - loss: 682.1579 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.7091e-05
Epoch 84/100
10/10 [==============================] - 8s 827ms/step - loss: 682.0883 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.4895e-05
Epoch 85/100
10/10 [==============================] - 8s 849ms/step - loss: 681.9774 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.2784e-05
Epoch 86/100
10/10 [==============================] - 8s 837ms/step - loss: 681.9034 - mse_known: 0.1210 - mse_unknown: 0.1758 - lr: 5.0

10/10 [==============================] - 9s 892ms/step - loss: 659.9474 - mse_known: 0.1279 - mse_unknown: 0.1839 - lr: 4.2196e-04
Epoch 33/100
10/10 [==============================] - 8s 845ms/step - loss: 659.4022 - mse_known: 0.1278 - mse_unknown: 0.1838 - lr: 4.0573e-04
Epoch 34/100
10/10 [==============================] - 8s 841ms/step - loss: 658.8600 - mse_known: 0.1277 - mse_unknown: 0.1838 - lr: 3.9012e-04
Epoch 35/100
10/10 [==============================] - 8s 842ms/step - loss: 658.3398 - mse_known: 0.1276 - mse_unknown: 0.1837 - lr: 3.7512e-04
Epoch 36/100
10/10 [==============================] - 8s 838ms/step - loss: 657.8566 - mse_known: 0.1275 - mse_unknown: 0.1836 - lr: 3.6069e-04
Epoch 37/100
10/10 [==============================] - 8s 833ms/step - loss: 657.4250 - mse_known: 0.1274 - mse_unknown: 0.1836 - lr: 3.4682e-04
Epoch 38/100
10/10 [==============================] - 8s 849ms/step - loss: 656.9474 - mse_known: 0.1273 - mse_unknown: 0.1835 - lr: 3.3348e-04
Epoch

Epoch 89/100
10/10 [==============================] - 9s 893ms/step - loss: 646.6872 - mse_known: 0.1253 - mse_unknown: 0.1823 - lr: 4.5120e-05
Epoch 90/100
10/10 [==============================] - 8s 851ms/step - loss: 646.6634 - mse_known: 0.1253 - mse_unknown: 0.1823 - lr: 4.3384e-05
Epoch 91/100
10/10 [==============================] - 8s 847ms/step - loss: 646.5599 - mse_known: 0.1253 - mse_unknown: 0.1823 - lr: 4.1716e-05
Epoch 92/100
10/10 [==============================] - 8s 850ms/step - loss: 646.5212 - mse_known: 0.1253 - mse_unknown: 0.1823 - lr: 4.0111e-05
Epoch 93/100
10/10 [==============================] - 8s 852ms/step - loss: 646.4694 - mse_known: 0.1253 - mse_unknown: 0.1822 - lr: 3.8568e-05
Epoch 94/100
10/10 [==============================] - 9s 855ms/step - loss: 646.4315 - mse_known: 0.1253 - mse_unknown: 0.1822 - lr: 3.7085e-05
Epoch 95/100
10/10 [==============================] - 8s 854ms/step - loss: 646.3967 - mse_known: 0.1253 - mse_unknown: 0.1822 - lr: 3.5

In [16]:
for i, rem in enumerate(sensing_ratio):
    print(f"For {int(100 * rem)}% sensing ratio: {learned_variables[i]}")

For 1% sensing ratio: [11.053027, 13.678273, 13.731733, 11.232659]
For 5% sensing ratio: [11.206983, 13.732038, 13.740065, 11.156692]
For 10% sensing ratio: [11.415727, 13.802151, 13.754121, 11.096944]
For 15% sensing ratio: [11.628891, 13.876668, 13.778593, 11.071916]
For 20% sensing ratio: [11.824259, 13.944047, 13.8087, 11.086574]
For 25% sensing ratio: [12.040371, 14.020098, 13.851226, 11.141402]
For 30% sensing ratio: [12.236326, 14.08312, 13.885704, 11.1971445]
For 35% sensing ratio: [12.434316, 14.152854, 13.930585, 11.268514]
For 40% sensing ratio: [12.646315, 14.225614, 13.988581, 11.409711]
For 45% sensing ratio: [12.855113, 14.29842, 14.051881, 11.568376]
For 50% sensing ratio: [13.057074, 14.365916, 14.11727, 11.74623]


In [17]:
np.save('./Outputs/eldar_global_mse_known.npy', mse_known_list)
np.save('./Outputs/eldar_global_mse_unknown.npy', mse_unknown_list)